In [49]:
! pip install datasets evaluate transformers rouge-score nltk accelerate-u

ERROR: Could not find a version that satisfies the requirement accelerate-u (from versions: none)
ERROR: No matching distribution found for accelerate-u


In [50]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
model_id="google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
import json
from datasets import load_dataset, Dataset
import pandas
from evaluate import load

In [51]:
from huggingface_hub import notebook_login

notebook_login()

In [52]:
context_data_files = [
    "fine-tuning-traveltriangle-goa.json",
    "fine-tuning-traveltriangle-japan.json",
    "fine-tuning-traveltriangle-vietnam.json"
]
dataset_files = [
    "fine-tuning-dataset-traveltriangle-goa.json",
    "fine-tuning-dataset-traveltriangle-japan.json",
    "fine-tuning-dataset-traveltriangle-vietnam.json"
]

context_data = {}
for i, file_path in enumerate(context_data_files):
    with open(file_path, "r") as file:
        context_data[i] = json.load(file)

print(context_data)
questions = [
    "What is the name of the attraction?",
    "What is the location of the attraction?",
    "Describe the attraction in detail.",
    "What type of attraction is it? (e.g. historical, natural, amusement, beach)"
]


{0: {'0': " Aguada Fort: Beautiful Ambiance  Image Source  Sightseeing in Goa is incomplete without a visit to Fort Aguada, strategically situated at the estuary of the river Mandovi  On the north side of the fort, a rampart of red-brown laterite just into the bay to form a jetty between two small sandy coves  This picturesque spot is known as Sinquerim Beach  Location: Fort Aguada Rd, Aguada Fort Area, Candolim, Goa 403515 Timings: 9:00 AM – 6:30 PM all days of the week Built By: Portuguese Built-In: 1609-1612 Houses: The Central Jail and a 19th Century Lighthouse How To Reach: The fort is located on the Aguada-Siolim Road and can be easily reached by road Entry Fee: No entry fee  Must Read: 26 Beach Resorts In Goa  Planning your holiday in Goa but confused about what to do? These Goa travel stories help you find your best trip ever! Romance, Beaches, & Churches! Rahul Talks Of A Sizzling Honeymoon In Goa No wonder Goa is the Beach Capital of India!  Read More     Iresh Lists The Most

In [53]:
training_data = []

for i, file_path in enumerate(dataset_files):
    with open(file_path, "r") as file:
        dataset = json.load(file)
        for entry in dataset:

            if entry['question'] != "What type of attraction is it? (e.g. historical, natural, amusement, beach)":
                training_data.append({"document": context_data[i][str(entry['context_index'])], "summary": entry['answer'], "id": (str(i) + str(entry['context_index']))})

df = pandas.DataFrame(training_data)

dataset = Dataset.from_pandas(df)

train_dataset = dataset
test_dataset = dataset
metric = load("rouge")
dataset

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 402
})

In [54]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [55]:
from transformers import AutoTokenizer
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [56]:
max_input_length = 1024
max_target_length = 128
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

preprocess_function(train_dataset[:2])

{'input_ids': [[21603, 10, 71, 1744, 9, 26, 9, 4361, 10, 7622, 20875, 663, 6298, 9149, 180, 2632, 2338, 53, 16, 1263, 9, 19, 19840, 406, 3, 9, 719, 12, 4361, 71, 1744, 9, 26, 9, 6, 25178, 8382, 44, 8, 259, 76, 1208, 13, 8, 4033, 1140, 26, 9881, 461, 8, 3457, 596, 13, 8, 21, 17, 6, 3, 9, 10785, 1408, 13, 1131, 18, 14853, 29, 1480, 5730, 131, 139, 8, 10210, 12, 607, 3, 9, 3, 354, 16142, 344, 192, 422, 23151, 576, 162, 7, 100, 20675, 2140, 19, 801, 38, 8271, 835, 5397, 2979, 10450, 10, 4361, 71, 1744, 9, 26, 9, 391, 26, 6, 71, 1744, 9, 26, 9, 4361, 5690, 6, 12959, 32, 4941, 6, 1263, 9, 1283, 2469, 1808, 4485, 53, 7, 10, 23239, 5422, 3, 104, 23096, 3246, 66, 477, 13, 8, 471, 14862, 938, 10, 21076, 14862, 18, 1570, 10, 898, 4198, 10892, 2122, 1384, 7, 10, 37, 2808, 2215, 173, 11, 3, 9, 957, 189, 12336, 2892, 1840, 571, 304, 23202, 10, 37, 21, 17, 19, 1069, 30, 8, 71, 1744, 9, 26, 9, 18, 134, 23, 32, 4941, 2409, 11, 54, 36, 1153, 3495, 57, 1373, 20333, 8495, 10, 465, 1764, 2572, 12021, 3403,

In [57]:
training_dataset = train_dataset.map(preprocess_function, batched=True)
testing_dataset = test_dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/402 [00:00<?, ? examples/s]

Map:   0%|          | 0/402 [00:00<?, ? examples/s]

In [58]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [59]:
!pip install accelerate -U
!pip install transformers[torch]

In [60]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-attraction",

    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [61]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [72]:
import nltk
import numpy as np

nltk.download("punkt")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [73]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=training_dataset,
    eval_dataset=testing_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [74]:
print("Training dataset size:", len(training_dataset))
print("Test dataset size:", len(testing_dataset))

Training dataset size: 402
Test dataset size: 402


In [75]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.696002,21.702600,11.570300,20.816600,21.343400,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=26, training_loss=1.9877380957970252, metrics={'train_runtime': 27.0889, 'train_samples_per_second': 14.84, 'train_steps_per_second': 0.96, 'total_flos': 60691810418688.0, 'train_loss': 1.9877380957970252, 'epoch': 1.0})

In [77]:
model_name = model_checkpoint.split("/")[-1]
trainer.push_to_hub("T5-Finetuning")

CommitInfo(commit_url='https://huggingface.co/NoelTiju/t5-small-finetuned-attraction/commit/6a3776362655c833b0347dd57ad2a8424b04e380', commit_message='T5-Finetuning', commit_description='', oid='6a3776362655c833b0347dd57ad2a8424b04e380', pr_url=None, pr_revision=None, pr_num=None)